Функции

In [ ]:
#Replacing flags
def Y_or_N(x):
  return [1,0][x.upper() == 'Y']

In [ ]:
#Converting days to years
def my_years_my_wealth(days):
  cur_date = datetime.datetime.now()
  #print(pd.to_datetime(days, origin=cur_date, unit='D').year)
  if days > -365:
    return 0
  return cur_date.year - pd.to_datetime(days, origin=cur_date, unit='D').year


In [ ]:
# Chart drawing function
def graffs(arr):
  fig, ax =plt.subplots(1,len(arr), figsize=(18,6))
  fig.autofmt_xdate()
  for index, category in enumerate(arr):
    
    #plt.subplot(1, index+1)   
    sns.countplot(data = df, x =category, hue='status',ax=ax[index])
    ax[index].set_title(category)
    bars = ax[index].patches
    half = int(len(bars)/2)
    left_bars = bars[:half]
    right_bars = bars[half:]

    for left, right in zip(left_bars, right_bars):
        height_l = 0 if np.isnan(right.get_height()) else left.get_height() #Высота левого бара
        height_r = 0 if np.isnan(right.get_height()) else right.get_height() #Высота правого бара
        total = height_l + height_r #общая сумма строк
        if total > 0:
          
          #sign left 
          ax[index].text(left.get_x() + left.get_width()/2., height_l + 50, '{0:.0%}'.format(height_l/total), ha="center")
          #ax[index].text(left.get_x() + left.get_width()/2., height_l + 350, str(height_l), ha="center")
          
          #sign right
          #ax[index].text(right.get_x() + right.get_width()/2., height_r + 350, str(height_r), ha="center")
          ax[index].text(right.get_x() + right.get_width()/2., height_r + 50, '{0:.0%}'.format(height_r/total), ha="center")
  plt.tight_layout()        
  plt.show()

In [ ]:
import pandas as pd
!pip install catboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import  GridSearchCV
from sklearn.linear_model import LogisticRegression
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (7,5)
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle
sns.set_theme()
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_curve, roc_auc_score
from catboost import  CatBoostClassifier
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

# Загрузка данных и формирование датасета

In [ ]:
appl = pd.read_csv("../input/credit-card-approval-prediction/application_record.csv")

In [ ]:
balance = pd.read_csv("../input/credit-card-approval-prediction/credit_record.csv")

In [ ]:
appl.columns  = [i.lower() for i in appl.columns]

In [ ]:
balance.columns = [i.lower() for i in balance.columns]

In [ ]:
appl.info()

In [ ]:
balance.info()

In [ ]:
appl.head(5)

In [ ]:
appl.describe()

In [ ]:
# Let's immediately remove the mobile sign, everyone has a mobile.
appl.drop('flag_mobil',axis=1,inplace=True)

In [ ]:
balance

In [ ]:
balance.describe()

In [ ]:
#Let's take the data of active clients who have a history from the current date (from the zero month) - an open loan
balance = balance[balance.id.isin(balance[balance.months_balance==0]['id'])]

In [ ]:
balance.groupby('status')['id'].count()

1.	Appl, contains general customer data. Most likely, these are primary data and are stored in the storage based on received applications for the issuance of loan products.

2.  Balance, here you can find historical data on issued loan products, basic information, payment schedule for each loan and payment status.

#Determination of the target characteristic

In [ ]:
# determination of the month of opening a loan (the expected date of issuing a loan) and the current month
credits_history = balance.groupby(['id']).agg(
   start_month=  ('months_balance', min), end_month=('months_balance', max), 
   current_status =('status', 'first'),
   last_status = ('status', 'last')
    ).reset_index()

In [ ]:
# the length of the credit history, we will calculate with rounding
credits_history['duration'] = round(abs(credits_history.start_month) / 12)

In [ ]:
# Clients who have a loan that has been issued closed no more than 6 months ago
#credits_history = credits_history[credits_history.start_month < -]

In [ ]:
# connect with statuses
credits_history = pd.merge(credits_history, balance, how='left', on='id')

In [ ]:
#credits_history.start_month = abs(credits_history.start_month) + credits_history.months_balance

In [ ]:
#Let's filter out closed loans. Status X - no credit for the current month
credits_history = credits_history[~credits_history.current_status.isin(["X"])]

In [ ]:
credits_history

In [ ]:
balance[balance.id==5001711]

Для анализа просроченных платежей посчитаем сколько и каких статусов в каждом месяце.

In [ ]:
# grouping and counting statuses by month
bad_month = credits_history.groupby(['months_balance', 'status']).size().reset_index(name='counts')

In [ ]:
bad_month

In [ ]:
# remove lines with good indicators
bad_month.drop(bad_month[bad_month.status.isin(["X","C"])].index,inplace=True)

In [ ]:
bad_month['months_balance'] =  abs(bad_month['months_balance'])

In [ ]:
bad_month

In [ ]:
# General status chart

sns.lineplot(data=bad_month, x='months_balance', y='counts', hue='status')

In [ ]:
# status chart without 0
sns.lineplot(data=bad_month[bad_month.status != '0'], x='months_balance', y='counts', hue='status')

Thus, a level 0 delay occurs in most clients, during the entire loan period, the peak falls on the first 5 months. Delay of the first level is also not uncommon to have a similar distribution pattern with zero, but with a lower frequency.

Overdue payments of levels 2, 3, 4, 5 Significantly fewer, but they are of a more prolonged nature. In long-term loans, users with a history of more than 5 years after about 50 months have no such delays at all.

Based on this data, we will mark up the dataset with general data as follows.

A debt of 1-5 level formed within 18 months is a "bad client"

In [ ]:
bad = ["1", "2", "3", "4", "5"]

In [ ]:
#The function of determining the debt at the client within 18 months
def chek(arr):
  s = 1
  for i,j in enumerate(arr):
    if i <= 17 and j in bad:
      s = 0
      break   
  return s  

In [ ]:
target = credits_history.groupby('id')['status'].apply(chek).reset_index()

In [ ]:
target[target.status == 1]

In [ ]:
# Share of good clients
target['status'].mean()

Получилось что доля "плохих" всего ~8%

In [ ]:
#Now let's connect to the target attribute with general data from the appl dataset and the received credit history
df = appl.merge(target, on='id', how='inner')

In [ ]:
df = df.merge(credits_history.groupby('id')[['duration','start_month']].first(), on='id', how='left')

In [ ]:
df.name_income_type.unique()

In [ ]:
# delete lines without a target attribute
df = df.drop(df[df.status.isna()].index)

#Data processing

## Skips

Fill in the gaps in occupation_type with the most popular amusement status


In [ ]:
df.info()

In [ ]:
df.groupby(['name_income_type','occupation_type']).count().sort_values('id').groupby(level=0).tail(1)['id']

In [ ]:
df.loc[df['name_income_type'] == 'Working','occupation_type']

In [ ]:
df.loc[df['name_income_type'] == 'Working','occupation_type'] = \
df.loc[df['name_income_type'] == 'Working','occupation_type'].fillna(value='Laborers')

In [ ]:
df.loc[df['name_income_type'] == 'Commercial associate','occupation_type'] = \
df.loc[df['name_income_type'] == 'Commercial associate','occupation_type'].fillna(value='Laborers')

In [ ]:
df.loc[df['name_income_type'] == 'State servant','occupation_type'] = \
df.loc[df['name_income_type'] == 'State servant','occupation_type'].fillna(value='Core staff')

In [ ]:
df.loc[df['name_income_type'] == 'Student','occupation_type'] = \
df.loc[df['name_income_type'] == 'Student','occupation_type'].fillna(value='Laborers')

In [ ]:
df.loc[df['name_income_type'] == 'Pensioner','occupation_type'] = \
df.loc[df['name_income_type'] == 'Pensioner','occupation_type'].fillna(value='Core staff')

In [ ]:
#df['occupation_type'] = df['occupation_type'].fillna(value='uncnown')

In [ ]:
df.info()

## Duplicates

Let's remove duplicates. Judging by the fact that we have identical records in all respects except for the duration of the loan, we can assume that these are two different loan products from the same client.
Since we have an expected date of issue of the loan, we can assign numbers to all products. Let's get a new sign with the number of credits.
But, there is an assumption that this feature differs from the primary data and in order to get it, you need to make an additional request, or add it to the application.


In [ ]:
len(df[df.status == 1])

In [ ]:
len(df[df.status == 0])

У нас сильный дисбаланс классов, 90/10

Проверим наличие дубликатов. 



Теперь проверим не попал ли клиент в обе категории и плохой и хороший, если да то удалим хорошего. Плохих и так мало 

In [ ]:

df[df.drop(['id'],axis=1).duplicated(keep=False)==True].groupby('status')['id'].count()

In [ ]:
df.drop(df[df.drop(['id'],axis=1).duplicated()==True].index, inplace=True)

Еще есть идентичные по всем параметрам записи, кроме пола

In [ ]:

df[df.drop(['id','code_gender'],axis=1).duplicated(keep=False)==True].sort_values(by='days_birth')

In [ ]:
df.drop(df[df.drop(['id','code_gender'],axis=1).duplicated(keep=False)==True].index,inplace=True)

In [ ]:
df['credit_counts'] = 1

In [ ]:
columns = df.drop(['id','credit_counts','status','duration','start_month'],axis=1).columns.tolist()

In [ ]:
df['credit_counts'] = df.groupby(columns)['credit_counts'].apply(np.cumsum)

In [ ]:
df.credit_counts.describe()

In [ ]:
df.credit_counts.hist()

In [ ]:
#df.drop(df[df.credit_counts > 5].index,inplace=True)

In [ ]:
df.groupby('status')['status'].count()


# Primary processing of features

---



In [ ]:
#Для удобства перевдем годовой доход в месячный
#df.amt_income_total = df.amt_income_total/12

In [ ]:

df['income_category'] = pd.qcut(df.amt_income_total,5,labels=[1, 2, 3, 4, 5])

In [ ]:

df['income_per_person'] = df.amt_income_total / df.cnt_fam_members

In [ ]:
 
df['days_employed'] = df.days_employed.apply(my_years_my_wealth)

In [ ]:

df['days_birth'] = df.days_birth.apply(my_years_my_wealth)

In [ ]:

df['age_when_start'] = df.days_birth - df.duration
df['emploed_when_start'] = df.days_employed - df.duration

In [ ]:
df.drop(columns=['days_birth','days_employed'], inplace=True)

In [ ]:
df

#Data visualization

In [ ]:
fig, ax =plt.subplots(1,3, figsize=(19,5))
sns.boxenplot(data=df, x='amt_income_total', hue='status',ax=ax[0])
sns.boxenplot(data=df, x='cnt_children', hue='status',ax=ax[1])
sns.boxenplot(data=df, x='cnt_fam_members', hue='status',ax=ax[2])
plt.tight_layout()  
plt.show()

In [ ]:

#df.drop(df[df.amt_income_total < 50000].index, inplace=True)
#df.drop(df[df.amt_income_total > 500000].index, inplace=True)
df.drop(df[df.cnt_children > 3].index, inplace=True)
df.drop(df[df.cnt_fam_members > 5].index, inplace=True)

In [ ]:
df.groupby('status')['status'].count()

In [ ]:

graffs(['code_gender','flag_own_realty','flag_own_car'])

In [ ]:

graffs(['name_family_status','name_income_type','name_education_type'])

In [ ]:

graffs(['name_housing_type','occupation_type'])

In [ ]:

fig, ax =plt.subplots(1,2, figsize=(19,5))
sns.histplot(data=df, x='age_when_start', hue='status',ax=ax[0])
sns.histplot(data=df, x='emploed_when_start', hue='status',ax=ax[1])
plt.tight_layout()  
plt.show()

In [ ]:

graffs(['cnt_children','cnt_fam_members'])

In [ ]:
fig, ax =plt.subplots(1,2, figsize=(27,7))
sns.histplot(data=df, x='income_category', hue='status',ax=ax[0])
sns.boxenplot(data=df, x='amt_income_total', hue='status',ax=ax[1])
plt.tight_layout()  
plt.show()

## Preliminary entry according to the schedules:
If we talk about clients, then, guided by the graphs, the portrait of an ideal client is approximately the following:

A woman aged 28, 32 or 38 years old, owning real estate (apartment or house) and a car. Of course, she is married, no children or a maximum of two (family of 4).
He works, about 6 years of experience, holds the position of a manager or a sales manager or an accountant with an income of more than 5,000 per month, at least has a secondary specialized education.

But all this is far from reality and cannot always correspond to reality.

Rational definitions of the user's portrait use binning. This is from credit scoring tasks to categorize variable binning procedures and obtain a WOE for each category.

#Dataset preparation

In [ ]:
df.credit_counts.describe()

In [ ]:
#
df.flag_own_car = df.flag_own_car.apply(Y_or_N)

In [ ]:
#If the experience at the time of application is less, we indicate -1
df.loc[df['emploed_when_start'] < 0, 'emploed_when_start'] = -1

In [ ]:
df.flag_own_realty = df.flag_own_realty.apply(Y_or_N)

In [ ]:
cm = sns.light_palette("green", as_cmap=True)
df.corr().style.background_gradient(cmap=cm)

In [ ]:
df.head(2)

In [ ]:
#Remove unnecessary data from the dataset
df_features = df.drop(['id','status','start_month','duration',
                       'amt_income_total'], axis=1)
df_target = df['status']

In [ ]:
num_features = ['income_category','age_when_start','emploed_when_start','cnt_fam_members','credit_counts','income_per_person']

In [ ]:
cat_features=['code_gender','name_income_type','name_education_type','name_family_status',
              'name_housing_type','occupation_type']

In [ ]:
df_features[num_features] = df_features[num_features].astype('int64')

In [ ]:
df.groupby('status')['status'].count()

In [ ]:
df_features.info()

# Model training

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, test_size=0.2, random_state=42)

In [ ]:
# upsample
def upsample(features, target, repeat):
    
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
 
    features_upsampled = pd.concat([features_ones] + [features_zeros] * repeat)
    target_upsampled = pd.concat([target_ones] + [target_zeros] * repeat)
    
    features_upsampled = shuffle(features_upsampled, random_state=42)
    target_upsampled = shuffle(target_upsampled, random_state=42)
    
    return features_upsampled, target_upsampled


In [ ]:
features_upsampled, target_upsampled = upsample(X_train, y_train, 8)
print(features_upsampled.shape)
print(target_upsampled.mean())

In [ ]:
# downsample
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_sample = features_ones.sample(frac=fraction, random_state=42)
    target_sample = target_ones.sample(frac=fraction, random_state=42)
    
    features_downsampled = pd.concat([features_sample] + [features_zeros])
    target_downsampled = pd.concat([target_sample] + [target_zeros])
    
    features_downsampled = shuffle(features_downsampled, random_state=42)
    target_downsampled = shuffle(target_downsampled, random_state=42)
    
    return features_downsampled, target_downsampled

In [ ]:
features_downsample, target_downsample = downsample(X_train, y_train, 0.17)
print(features_downsample.shape)
print(target_downsample.mean())

In [ ]:
def results(model, scor_type, X, y, sample):
  predict = model.predict(X)
  tn, fp, fn, tp = confusion_matrix(y, predict).ravel()
  f1 = f1_score(y, predict)
  roc = roc_auc_score(y, model.predict_proba(X)[:,1])
  f_beta = fbeta_score(y, predict,beta=0.5)
  return ({'dataset': sample,
                      'tune_metric': scor_type,
                      'roc_auc_score': roc,
                      'f1_beta_score': f1,
                      'f1_score': f1,
                      'tn': tn,
                      'fp': fp,
                      'fn': fn,
                      'tp': tp})

In [ ]:
#Датасет для результатов
columns = ['dataset','tune_metric','roc_auc_score','f1_beta_score','f1_score', 'tn','fp','fn','tp']

In [ ]:
res_metrics = pd.DataFrame(columns=columns)

In [ ]:

categories = []
for cat in cat_features:
  categories.append(df[cat].unique().tolist())

## pipeline

In [ ]:
preprocessor = ColumnTransformer(
    [
        ("onehot_categorical", OneHotEncoder(categories=categories, drop='first'), cat_features),
        ('scaler', StandardScaler(), num_features)
    ], remainder='passthrough'
)

In [ ]:
pipeline = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('model', LogisticRegression())
    ]
)


In [ ]:
preprocessor.fit_transform(features_downsample)[:1]

In [ ]:
params = [
  {
        'model': [RandomForestClassifier()],
        'model__n_estimators': [i for i in range(1, 70,5)],
        'model__max_depth': [i for i in range(1, 20,2)],
        'model__random_state': [42],
        'model__criterion':['gini', 'entropy'],
        'preprocessor__scaler': ['passthrough']
      
        
  },
  {   
        'model': [LogisticRegression()],   
        'model__solver': ['liblinear'],
        #'model__penalty': ['l1', 'l2'],
        #'model__C': [0.01, 0.1, 1, 10],
        'model__random_state': [42],
        'preprocessor__scaler': [StandardScaler()]
   
  }

]

In [ ]:
grid = GridSearchCV(pipeline, param_grid=params, verbose=False,  
                                                 cv=3, 
                                                 scoring='roc_auc')

In [ ]:
grid.fit(features_upsampled, target_upsampled)

In [ ]:
pd.DataFrame(grid.cv_results_).sort_values(by="mean_test_score", ascending=False)[[
                                                                                  'param_model__max_depth',
                                                                                  'param_model__n_estimators',
                                                                                  'mean_test_score',
                                                                                  'param_model']].head(3)

In [ ]:
grid.scorer_

In [ ]:
res_metrics = res_metrics.append(results(grid.best_estimator_, grid.scorer_,
                                         X_test,y_test,
                                         'test/upsampled'),ignore_index=True)


In [ ]:
res_metrics

In [ ]:
probabilities_valid = grid.best_estimator_.predict_proba(X_test)

In [ ]:
probabilities_one_test = probabilities_valid[:, 1]

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, probabilities_one_test) 

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, linestyle='-')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')

In [ ]:

for threshold in np.arange(0, 0.95, 0.05):
    predicted_valid =  (probabilities_one_test > threshold)*1
    precision = precision_score(y_test,predicted_valid)
    recall = recall_score(y_test,predicted_valid)
    tn, fp, fn, tp = confusion_matrix(y_test, predicted_valid).ravel()
    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f}".format(
        threshold, precision, recall))
    print('TN = {}| FP {}| FN {} | TP {} |'.format(tn, fp, fn, tp))

In [ ]:
cbr = CatBoostClassifier(eval_metric='AUC', iterations=10000, random_state=42,
                         learning_rate=0.05, depth=6, verbose=1000)


In [ ]:
cbr.fit(features_upsampled ,target_upsampled, cat_features=cat_features , eval_set=(X_test, y_test), plot=True)


In [ ]:
catboost_feachures_important=pd.DataFrame({'feature_importance': cbr.feature_importances_, 
              'feature_names': X_train.columns}).sort_values(by=['feature_importance'], 
                                                           ascending=False)

In [ ]:
catboost_feachures_important

In [ ]:
res_metrics = res_metrics.append(results(cbr, 'roc_auc',
                                         X_test,y_test,
                                         'test/upsampled'),ignore_index=True)

In [ ]:
res_metrics

In [ ]:
probabilities_valid = cbr.predict_proba(X_test)

In [ ]:
probabilities_one_test = probabilities_valid[:, 1]

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, probabilities_one_test) 

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, linestyle='-')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')

# Result




dataset|	tune_metric|roc_auc_score|f1_beta_score|f1_score| tn | fp | fn |tp 
-------|------------|-------------|-------------|--------|----|----|----|---
 test/upsampled RFC   | roc_auc_score |	0.603001 |0.937789 |0.937789|	47 | 300 | 130| 3241 
test/upsampled   Catboost  |	roc_auc|	0.610053|	0.915103|	0.915103|	76|	271|	299|	3072|
test/downsample RFC|roc_auc_score|	0.600216|	0.907155|	0.907155|	75|	272|	347|	3024
test/downsample Catboost|roc_auc	|0.590445|	0.888004|	0.888004|	89|	258|	473|	2898|
test/X_train RFC|	roc_auc_score|	0.597396|	0.950820|	0.950820|	6|341|7|	3364|
test/X_train Catboost|	roc_auc|	0.590898|	0.951102|	0.951102|	7	|340|	6	|3365|

1. Develop a model for finding a “good” customer by credit cards


---
one.
Training and selection of hyperparameters for models was carried out using GridSearchCV

3 models trained

LogisticRegression

RandomForestClassifier

Catboost


2. Interpretation of model results, description of a "good" client and its probabilistic meaning


---
Choosing a metric. The main metric for fitting the model was chosen roc_auc, it will reflect well the "skill" of the model in ranking objects. The second point why this metric was chosen is that the model must have a probabilistic meaning. In this case, the indicator of the roc_auc metric can be interpreted as the probability with which a good customer will be classified as good.

Precision with such an imbalance of classes and a predominance of a positive class as a whole will be high if we mark all clients as good, the indicator will also be high, recall in this situation will be 1 (if we can still change the training dataset, but the test dataset will be with the same imbalance). F1 measure is based on precision / recall. But for comparison, the indicator of this metric was also calculated.

---




It was not possible to properly define a good client, in my opinion the model does not classify clients well. Its indicator roc_auc ** 0.61% ** indicates that with a probability of * 61% * the model is able to classify a good client as a “good customer”. The random model has an indicator of 50%, it turns out that ours is not so far "gone." Then make a decision



It turned out to be a "conditional" description of the client based on the data of their graphic interpretation, but these are only hypotheses.

A woman, 28, 32 or 38 years old, who owns real estate (apartment or house) and a car. Of course, she is married, no children or a maximum of two (family of 4). He works, about 6 years of experience, holds the position of a manager, sales manager or accountant, with an income of more than 5000 per month, secondary specialized education.


It is more rational to use the binning procedure to determine the user's portrait. This is from credit scoring tasks to categorize variables by binning and obtain a WOE for each category.
